### Differential Protein Expression Analysis

This section analyzes the exosome proteomic data to identify significantly dysregulated proteins in endometrial cancer.

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

# Load the proteomic data
proteomic_data = pd.read_csv('exosome_proteomic_data.csv')

# Separate the data into EC and control groups
ec_data = proteomic_data[proteomic_data['group'] == 'EC']
control_data = proteomic_data[proteomic_data['group'] == 'Control']

# Perform differential expression analysis
p_values = []
fold_changes = []
proteins = []
for protein in proteomic_data.columns[2:]:  # Assuming first two columns are sample and group
    ec_mean = ec_data[protein].mean()
    control_mean = control_data[protein].mean()
    fold_change = ec_mean / control_mean
    fold_changes.append(fold_change)
    t_stat, p_val = stats.ttest_ind(ec_data[protein], control_data[protein], equal_var=False)
    p_values.append(p_val)
    proteins.append(protein)

# Create a results dataframe
results = pd.DataFrame({
    'Protein': proteins,
    'FoldChange': fold_changes,
    'PValue': p_values
})

# Adjust p-values for multiple testing
results['AdjPValue'] = stats.multitest.multipletests(results['PValue'], method='fdr_bh')[1]

# Filter significantly dysregulated proteins (e.g., AdjPValue < 0.05 and |log2FC| > 1)
significant_results = results[(results['AdjPValue'] < 0.05) & (np.abs(np.log2(results['FoldChange'])) > 1)]

# Save the significant results
significant_results.to_csv('significant_proteins.csv', index=False)

### Pathway Enrichment Analysis

Using the list of significantly dysregulated proteins, perform pathway enrichment to identify affected metabolic and catabolic pathways.

In [ ]:
from gseapy import enrichr

genes = significant_results['Protein'].tolist()

enr = enrichr(gene_list=genes,
              gene_sets='KEGG_2019_Human',
              organism='Human',
              outdir='enrichment_results',
              cutoff=0.05)

enr.results.head()

### Visualization of Dysregulated Pathways

Plot the top enriched metabolic pathways associated with endometrial cancer.

In [ ]:
enrichment_results = pd.read_csv('enrichment_results/KEGG_2019_Human.enrichr.reports.txt', sep='	')

top_paths = enrichment_results.nsmallest(10, 'P-value')

plt.figure(figsize=(10,6))
sns.barplot(x='Combined Score', y='Term', data=top_paths)
plt.title('Top Enriched Metabolic Pathways in Endometrial Cancer')
plt.xlabel('Combined Score')
plt.ylabel('Pathway')
plt.tight_layout()
plt.show()

### Summary of Findings

The analysis identifies key metabolic pathways that are dysregulated in endometrial cancer, providing potential targets for therapeutic intervention.





***
### [**Evolve This Code**](https://biologpt.com/?q=Evolve%20Code%3A%20This%20code%20performs%20differential%20protein%20expression%20analysis%20and%20pathway%20enrichment%20for%20exosome%20proteomic%20data%20from%20endometrial%20cancer%20patients%20and%20controls.%0A%0AIncorporate%20validation%20steps%20using%20independent%20datasets%20and%20expand%20pathway%20analysis%20to%20include%20additional%20databases%20for%20comprehensive%20insights.%0A%0AExosome%20proteomic%20profiling%20endometrial%20cancer%20metabolic%20pathways%20dysregulation%0A%0A%23%23%23%20Differential%20Protein%20Expression%20Analysis%0A%0AThis%20section%20analyzes%20the%20exosome%20proteomic%20data%20to%20identify%20significantly%20dysregulated%20proteins%20in%20endometrial%20cancer.%0A%0Aimport%20pandas%20as%20pd%0Aimport%20numpy%20as%20np%0Afrom%20scipy%20import%20stats%0Aimport%20matplotlib.pyplot%20as%20plt%0Aimport%20seaborn%20as%20sns%0A%0A%23%20Load%20the%20proteomic%20data%0Aproteomic_data%20%3D%20pd.read_csv%28%27exosome_proteomic_data.csv%27%29%0A%0A%23%20Separate%20the%20data%20into%20EC%20and%20control%20groups%0Aec_data%20%3D%20proteomic_data%5Bproteomic_data%5B%27group%27%5D%20%3D%3D%20%27EC%27%5D%0Acontrol_data%20%3D%20proteomic_data%5Bproteomic_data%5B%27group%27%5D%20%3D%3D%20%27Control%27%5D%0A%0A%23%20Perform%20differential%20expression%20analysis%0Ap_values%20%3D%20%5B%5D%0Afold_changes%20%3D%20%5B%5D%0Aproteins%20%3D%20%5B%5D%0Afor%20protein%20in%20proteomic_data.columns%5B2%3A%5D%3A%20%20%23%20Assuming%20first%20two%20columns%20are%20sample%20and%20group%0A%20%20%20%20ec_mean%20%3D%20ec_data%5Bprotein%5D.mean%28%29%0A%20%20%20%20control_mean%20%3D%20control_data%5Bprotein%5D.mean%28%29%0A%20%20%20%20fold_change%20%3D%20ec_mean%20%2F%20control_mean%0A%20%20%20%20fold_changes.append%28fold_change%29%0A%20%20%20%20t_stat%2C%20p_val%20%3D%20stats.ttest_ind%28ec_data%5Bprotein%5D%2C%20control_data%5Bprotein%5D%2C%20equal_var%3DFalse%29%0A%20%20%20%20p_values.append%28p_val%29%0A%20%20%20%20proteins.append%28protein%29%0A%0A%23%20Create%20a%20results%20dataframe%0Aresults%20%3D%20pd.DataFrame%28%7B%0A%20%20%20%20%27Protein%27%3A%20proteins%2C%0A%20%20%20%20%27FoldChange%27%3A%20fold_changes%2C%0A%20%20%20%20%27PValue%27%3A%20p_values%0A%7D%29%0A%0A%23%20Adjust%20p-values%20for%20multiple%20testing%0Aresults%5B%27AdjPValue%27%5D%20%3D%20stats.multitest.multipletests%28results%5B%27PValue%27%5D%2C%20method%3D%27fdr_bh%27%29%5B1%5D%0A%0A%23%20Filter%20significantly%20dysregulated%20proteins%20%28e.g.%2C%20AdjPValue%20%3C%200.05%20and%20%7Clog2FC%7C%20%3E%201%29%0Asignificant_results%20%3D%20results%5B%28results%5B%27AdjPValue%27%5D%20%3C%200.05%29%20%26%20%28np.abs%28np.log2%28results%5B%27FoldChange%27%5D%29%29%20%3E%201%29%5D%0A%0A%23%20Save%20the%20significant%20results%0Asignificant_results.to_csv%28%27significant_proteins.csv%27%2C%20index%3DFalse%29%0A%0A%23%23%23%20Pathway%20Enrichment%20Analysis%0A%0AUsing%20the%20list%20of%20significantly%20dysregulated%20proteins%2C%20perform%20pathway%20enrichment%20to%20identify%20affected%20metabolic%20and%20catabolic%20pathways.%0A%0Afrom%20gseapy%20import%20enrichr%0A%0Agenes%20%3D%20significant_results%5B%27Protein%27%5D.tolist%28%29%0A%0Aenr%20%3D%20enrichr%28gene_list%3Dgenes%2C%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20gene_sets%3D%27KEGG_2019_Human%27%2C%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20organism%3D%27Human%27%2C%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20outdir%3D%27enrichment_results%27%2C%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20cutoff%3D0.05%29%0A%0Aenr.results.head%28%29%0A%0A%23%23%23%20Visualization%20of%20Dysregulated%20Pathways%0A%0APlot%20the%20top%20enriched%20metabolic%20pathways%20associated%20with%20endometrial%20cancer.%0A%0Aenrichment_results%20%3D%20pd.read_csv%28%27enrichment_results%2FKEGG_2019_Human.enrichr.reports.txt%27%2C%20sep%3D%27%09%27%29%0A%0Atop_paths%20%3D%20enrichment_results.nsmallest%2810%2C%20%27P-value%27%29%0A%0Aplt.figure%28figsize%3D%2810%2C6%29%29%0Asns.barplot%28x%3D%27Combined%20Score%27%2C%20y%3D%27Term%27%2C%20data%3Dtop_paths%29%0Aplt.title%28%27Top%20Enriched%20Metabolic%20Pathways%20in%20Endometrial%20Cancer%27%29%0Aplt.xlabel%28%27Combined%20Score%27%29%0Aplt.ylabel%28%27Pathway%27%29%0Aplt.tight_layout%28%29%0Aplt.show%28%29%0A%0A%23%23%23%20Summary%20of%20Findings%0A%0AThe%20analysis%20identifies%20key%20metabolic%20pathways%20that%20are%20dysregulated%20in%20endometrial%20cancer%2C%20providing%20potential%20targets%20for%20therapeutic%20intervention.%0A%0A)
***

### [Created with BioloGPT](https://biologpt.com/?q=Paper%20Review%3A%20A%20Pilot%20Study%20of%20Exosome%20Proteomic%20Profiling%20Reveals%20Dysregulated%20Metabolic%20Pathways%20in%20Endometrial%20Cancer.)
[![BioloGPT Logo](https://biologpt.com/static/icons/bioinformatics_wizard.png)](https://biologpt.com/)
***